In [1]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle datasets download -d salader/dogs-vs-cats
!unzip -q dogs-vs-cats.zip -d dogs_vs_cats_data

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
 98% 1.04G/1.06G [00:08<00:00, 178MB/s]
100% 1.06G/1.06G [00:08<00:00, 136MB/s]


In [4]:
from torchvision.datasets import CIFAR10
from torchvision.utils import save_image
from torchvision import transforms
import os
os.makedirs("/content/dogs_vs_cats_data/train/other", exist_ok=True)
os.makedirs("/content/dogs_vs_cats_data/test/other", exist_ok=True)

from torchvision.datasets import CIFAR10
from torchvision.utils import save_image

# Load CIFAR10
cifar = CIFAR10(root="./", train=True, download=True)

# Save 200 'other' images (excluding cat=3, dog=5)
count = 0
for i, (img, label) in enumerate(cifar):
    if label not in [3, 5]:  # Skip cat(3) and dog(5)
        save_image(transforms.ToTensor()(img), f"/content/dogs_vs_cats_data/train/other/img_{count}.jpg")
        count += 1
    if count >= 200:
        break


# Load CIFAR10 test set
cifar_test = CIFAR10(root="./", train=False, download=True)

# Save 100 'other' images to test set
count = 0
for i, (img, label) in enumerate(cifar_test):
    if label not in [3, 5]:  # skip cat (3) and dog (5)
        save_path = f"/content/dogs_vs_cats_data/test/other/other_{count}.jpg"
        save_image(transforms.ToTensor()(img), save_path)
        count += 1
    if count >= 100:
        break

In [6]:
import os
import torch
import torchvision
import torch.optim as optim
from torchvision import datasets, transforms

from torch.utils.data import DataLoader
import torchvision.models as models
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

Using: cuda


In [7]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

train_dir = "/content/dogs_vs_cats_data/train"
test_dir = "/content/dogs_vs_cats_data/test"

train_ds = datasets.ImageFolder(train_dir, transform=transform)
val_ds = datasets.ImageFolder(test_dir, transform=transform)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)

In [10]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1), nn.ReLU(),
            nn.BatchNorm2d(32), nn.MaxPool2d(2), nn.Dropout(0.25),

            nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(),
            nn.BatchNorm2d(64), nn.MaxPool2d(2), nn.Dropout(0.25),

            nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(),
            nn.BatchNorm2d(128), nn.MaxPool2d(2), nn.Dropout(0.25),

            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 128), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, 3)
        )

    def forward(self, x):
        return self.net(x)

In [11]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [12]:
epochs = 20
best_val_loss = float('inf')
patience = 3
counter = 0

for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct_train = 0
    total_train = 0

    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Train accuracy
        _, preds = torch.max(outputs, 1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)

    train_loss = total_loss / len(train_loader)
    train_acc = correct_train / total_train

    # Validation
    model.eval()
    val_loss = 0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            val_loss += criterion(outputs, labels).item()

            _, preds = torch.max(outputs, 1)
            correct_val += (preds == labels).sum().item()
            total_val += labels.size(0)

    val_loss /= len(val_loader)
    val_acc = correct_val / total_val

    print(f"\nEpoch {epoch+1}/{epochs}")
    print(f"Train     -> Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}")
    print(f"Validation-> Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "cat_dog_other.pth")
        print("✔️ Model improved and saved.")
        counter = 0
    else:
        counter += 1
        print(f"⚠️ No improvement. Early stop counter: {counter}/{patience}")
        if counter >= patience:
            print("⛔ Early stopping.")
            break


100%|██████████| 632/632 [01:10<00:00,  8.95it/s]



Epoch 1/20
Train     -> Loss: 0.7591, Accuracy: 0.6130
Validation-> Loss: 0.6447, Accuracy: 0.6825
✔️ Model improved and saved.


100%|██████████| 632/632 [01:12<00:00,  8.75it/s]



Epoch 2/20
Train     -> Loss: 0.5794, Accuracy: 0.7135
Validation-> Loss: 0.5286, Accuracy: 0.7625
✔️ Model improved and saved.


100%|██████████| 632/632 [01:10<00:00,  8.93it/s]


...


Epoch 9/20
Train     -> Loss: 0.1930, Accuracy: 0.9208
Validation-> Loss: 0.3322, Accuracy: 0.8682
⚠️ No improvement. Early stop counter: 1/3


100%|██████████| 632/632 [01:10<00:00,  9.02it/s]



Epoch 10/20
Train     -> Loss: 0.1679, Accuracy: 0.9351
Validation-> Loss: 0.3691, Accuracy: 0.8704
⚠️ No improvement. Early stop counter: 2/3


100%|██████████| 632/632 [01:09<00:00,  9.03it/s]



Epoch 11/20
Train     -> Loss: 0.1633, Accuracy: 0.9386
Validation-> Loss: 0.3847, Accuracy: 0.8586
⚠️ No improvement. Early stop counter: 3/3
⛔ Early stopping.


In [8]:

def count_images_in_folder(folder):
    total = 0
    class_counts = {}
    for class_name in os.listdir(folder):
        class_path = os.path.join(folder, class_name)
        if os.path.isdir(class_path):
            count = len([f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
            class_counts[class_name] = count
            total += count
    return total, class_counts

# Count for train
train_total, train_classes = count_images_in_folder(train_dir)
print(f"\n📂 Train Total Images: {train_total}")
for label, count in train_classes.items():
    print(f"  └── {label}: {count}")

# Count for test
test_total, test_classes = count_images_in_folder(test_dir)
print(f"\n📂 Test Total Images: {test_total}")
for label, count in test_classes.items():
    print(f"  └── {label}: {count}")


📂 Train Total Images: 20200
  └── other: 200
  └── cats: 10000
  └── dogs: 10000

📂 Test Total Images: 5100
  └── other: 100
  └── cats: 2500
  └── dogs: 2500


In [9]:
print("Classes:", train_ds.classes)



Classes: ['cats', 'dogs', 'other']
